In [25]:
import pandas as pd

In [26]:
import os

homepath=os.path.expanduser("~")

In [27]:
df = pd.read_csv(f"{homepath}/code/fish_test_data.csv")

In [28]:
df.tail()

,Length,Weight,Label
9999995,34.636377,554.759082,Bream
9999996,13.172821,17.191087,Smelt
9999997,12.877214,15.103945,Smelt
9999998,39.071152,379.180396,Bream
9999999,32.942495,732.028292,Bream


In [29]:
sdf = df.sample(100000)          #sample하면 df에 있는 데이터 10만개 추출 랜덤으로.

In [30]:
sdf.to_csv(f"{homepath}/code/fish_r.data_100k.csv")        #데이터 저장  여기까지 10만개 추출

In [ ]:
# 아래부터 load

In [42]:
# load_csv
def load():   # airflow하려면 함수설정.
    import pandas as pd              
    df =pd.read_csv(f"{homepath}/code/fish_r.data_100k.csv")
    d = {
        "Bream":"도미",
        "Smelt":"빙어"
        }
    
    df["ko"]=df["Label"].apply(lambda x:d[x])

df

,Unnamed: 0,Length,Weight,Label,ko
0,2436005,35.717165,383.561583,Bream,도미
1,4929056,37.948401,383.706212,Bream,도미
2,1330716,30.483724,284.234613,Bream,도미
3,6203910,14.450491,9.772069,Smelt,빙어
4,1538118,10.988702,9.072952,Smelt,빙어
...,...,...,...,...,...
99995,5126832,31.569898,700.097738,Bream,도미
99996,9988326,35.730504,453.122278,Bream,도미
99997,7774321,10.745778,14.579903,Smelt,빙어
99998,1991773,33.064392,760.933881,Bream,도미


In [ ]:
# 아래부터 predict 시작

In [ ]:
""" 에어플로우에서 n값 인자로 넘겨주는거 , aiwflow 
op_kwargs={
    "n" =1
}
op_kwargs={
    "n" =5
}
"""


# 예측(requests)(도미인지 빙어인지)
def predict(n):
    import requests

    tmp = []

    for i, data in df.iterrows():     #데이터프레임에있는거를 한줄씩 반복한다.(iterrows) i는 순번
#    print(i, data)
        resp = requests.get(f"http://172.31.34.167/fish?length={data['Length']}&weight={data['Weight']}&neighbors={n}").text
        tmp.append(eval(resp)["prediction"])     # 빙어인지 도미인지 필요한 데이터 칼럼  필요함 , 딕셔너리 형태를 가져오려고 eval씀

    df[f"pred{n}"] = tmp

In [ ]:
for i, data in df.head().iterrows():
    print(i)
    print(data["Length"])
    print(data["Weight"])
    print(data["Label"])             # 한줄씩 반복하면서 따로 불러올수 있다. 

In [ ]:
for i, data in df.head().iterrows():   
#    print(i, data)
    resp = requests.get(f"http://15.206.80.113:81/fish?length={data['Length']}&weight={data['Weight']}&neighbors=1").text
    print(resp)

In [32]:
df["pred1"] = tmp #  tmp리스트에 빈값의 한 열을 추가함 , 데이터프레임에 도미인지 빙어인지 eval을 통해서 나온 tmp 값을 새로운 pred1의 칼럼에 추가
                # df는  위의 데이터프레임이다

NameError: name 'tmp' is not defined

In [ ]:
# 아래부터 agg(집계) 시작

In [ ]:
#혼동행열 쓰기
from sklean.metrics import confusion_matrix

# confusion.martrix(df["Label"],df["pred1"]     #혼동행열 실제 데이트 label, 예측 데이터 pred1 , 첫번째 실제, 두번째 예측 (열별로)

sum(df["Label"] == df["pred1"]) /len(df["pred1"])    # 정확도 

In [43]:
a=lambda x:x+1

In [44]:
a(1)

2